### Clase 1: Introducción al cómputo en la nube

En esta clase veremos brevemente sobre lo que es y no es el cómputo en la nube. También, aprenderás a hacer tu cuenta en Azure y desplegarás tu primer modelo de Machine Learning en la nube.

Este archivo corresponde a esta última parte.

## 1.1 Experimentos de Machine Learning.

A continuación, presentamos un experimento pequeño sobre Machine Learning.

Entonces los guardamos con el formato Pickle (ADVERTENCIA, NO ES SEGURO)

# DESPLEGANDO EN LA NUBE.

Lo primero que debemos hacer es generar un espacio de trabajo (workspace), utilizando para ello el id de nuestra suscripción.

In [1]:
#GETTING MY ID:
import json

id = open('my_id.json', 'r')
mi = json.load(id)

from azureml.core import Workspace

ws = Workspace.create(name="testmodel",
                      subscription_id = mi["my_id"],
                      resource_group = "deploymodel",
                      location = "centralindia") #centralindia

ModuleNotFoundError: No module named 'azureml'

Posteriormente, registramos el modelo que vamos a utilizar. En este caso, el modelo en formato pickle. También hay que asignarle un nombre.

In [ ]:
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="classifier",
                                  workspace=ws)

Registering model modelreg


In [7]:
from azureml.core.environment import Environment
virtual_env = Environment("env")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

In [8]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="API/score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name='testmodel',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\Felip\AppData\Local\Temp\ipykernel_27584\979962562.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [9]:
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


In [11]:
scoring_uri = service.scoring_uri
# Save the uri to a file
with open('scoring_uri.txt', 'w') as f:
    f.write(scoring_uri)


## USING THE MODEL

In [16]:
import pandas as pd
import json
input_data = pd.read_csv("load_data/temp_data/output.csv")
    # Serialize the first 5 rows of the DataFrame to JSON
input_data = input_data.head(5).to_json(orient="records")
input_data = {
    "data": json.loads(input_data)  # Asegúrate de que sea una lista de registros
}

model_input = json.dumps(input_data)

headers = {"Content-Type": "application/json"}
# ===================AQUI VA EL URI==================
scoring_uri = open('scoring_uri.txt', 'r').readline().strip()
# ====================AQUI VA EL URI==================
import requests
response = requests.post(scoring_uri, 
                         data=model_input, 
                         headers=headers)

if response.status_code == 200:
  result = json.loads(response.json())
  print(result)
  input_data["median_house_value"] = result["result"]
  display(pd.DataFrame(input_data["data"]))
else:
  print(f"Error: {response.text}")

{'result': [-0.03427901583899695, 0.8660552324617822, 0.05060939437120193, 0.8801751053895741, 0.9509326342492445]}


,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
0,1,False,Mr.,Orlando,N.,Gee,None,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2005-08-01 00:00:00.000
1,2,False,Mr.,Keith,None,Harris,None,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2006-08-01 00:00:00.000
2,3,False,Ms.,Donna,F.,Carreras,None,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2005-09-01 00:00:00.000
3,4,False,Ms.,Janet,M.,Gates,None,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2006-07-01 00:00:00.000
4,5,False,Mr.,Lucy,None,Harrington,None,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2006-09-01 00:00:00.000
